In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal
import cv2

import os
import random

In [2]:
path = '/home/peter/Documents/data/input/'
path_out = '/home/peter/Documents/data/input_converted/'

In [3]:
# go through file system
pet = []
ct = []

for r, d, f in os.walk(path):
    for file in f:
        if '.nii.gz' in file:
            if 'pet' in file:
                pet.append(os.path.join(r, file))
            else:
                ct.append(os.path.join(r, file))
print(pet[0])

/home/peter/Documents/data/input/20z1/8_pet_tk_it_nac.nii.gz


In [ ]:
# SAVE IN NEW FOLDER STRUCTURE ------------------------------------------------------------------------------------------

In [71]:
ref_size = (171, 200, 200)

In [ ]:
# save pet images (third dimension must match)

for p in pet:
    # load image
    orig_nii = sitk.ReadImage(p)
    orig = sitk.GetArrayFromImage(orig_nii)
    print(orig.shape)
    out = np.zeros(ref_size)
    
    # check size
    if orig.size != ref_size: 
        for s in range(171): #orig.shape[0]
            out[s,:,:] = cv2.resize(orig[s,:,:], ref_size[1:])
            
    # convert pack to NIFTI
    itkimage = sitk.GetImageFromArray(out, isVector=False)
    
    # copy metadata
    for k in orig_nii.GetMetaDataKeys():
        itkimage.SetMetaData(k, orig_nii.GetMetaData(k))
    itkimage.SetMetaData('dim[1]', str(ref_size[1]))
    itkimage.SetMetaData('dim[2]', str(ref_size[2]))
    itkimage.SetMetaData('dim[3]', str(ref_size[0]))
    
    # new filename
    filename = p.split('/')[-2] + '_pet.nii.gz'
    
    # save to new folder
    sitk.WriteImage(itkimage, os.path.join(path_out, filename), True)

In [ ]:
# save ct images

for p in ct:
    # load image
    orig_nii = sitk.ReadImage(p)
    orig = sitk.GetArrayFromImage(orig_nii)
    print(orig.shape)
    out = np.zeros(ref_size)
    
    # check size
    if orig.size != ref_size: 
        for s in range(171): #orig.shape[0]
            out[s,:,:] = cv2.resize(orig[s,:,:], ref_size[1:])
            
    # convert pack to NIFTI
    itkimage = sitk.GetImageFromArray(out, isVector=False)
    
    # copy metadata
    for k in orig_nii.GetMetaDataKeys():
        itkimage.SetMetaData(k, orig_nii.GetMetaData(k))
    itkimage.SetMetaData('dim[1]', str(ref_size[1]))
    itkimage.SetMetaData('dim[2]', str(ref_size[2]))
    itkimage.SetMetaData('dim[3]', str(ref_size[0]))
    
    # new filename
    filename = p.split('/')[-2] + '_ct.nii.gz'
    
    # save to new folder
    sitk.WriteImage(itkimage, os.path.join(path_out, filename), True)

In [ ]:
# SAVE AS NUMPY FILES ---------------------------------------------------------------------------------------------------

In [4]:
# select test set
testset_size = 8

dataset = [x.split('/')[-2] for x in pet]
testset = random.choices(dataset,k=testset_size)
trainset = [x for x in dataset if x not in testset]
print(testset)

['24z1', '05z9', '05z3', '15z1', '12z4', '12z3', '11z1', '07z4']


In [ ]:
pet_final = np.zeros((len(trainset)*171, 200, 200))
ct_final = np.zeros((len(trainset)*171, 200, 200))

for idx, i in enumerate(trainset):
    print(i)
    pet_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path_out, i + '_pet.nii.gz')))
    ct_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path_out, i + '_ct.nii.gz')))
    for x in range(pet_img.shape[0]):
        pet_final[idx*171+x,:,:] = pet_img[x,:,:]
        ct_final[idx*171+x,:,:] = ct_img[x,:,:]
    del pet_img, ct_img
    
np.save('/home/peter/Documents/data/pet.npy', pet_final)
np.save('/home/peter/Documents/data/ct.npy', ct_final)

In [7]:
# save image name lists to text files
with open('/home/peter/Documents/data/train.txt', 'w') as f:
    for item in trainset:
        f.write("%s\n" % item)

with open('/home/peter/Documents/data/test.txt', 'w') as f:
    for item in testset:
        f.write("%s\n" % item)